In [ ]:
#import library

In [5]:
import pandas as pd 
from bertopic import BERTopic 
from sentence_transformers import SentenceTransformer, util 
from umap import UMAP 
from hdbscan import HDBSCAN 
from sklearn.feature_extraction.text import CountVectorizer 
from bertopic.vectorizers import ClassTfidfTransformer 
import numpy as np 
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import re
import json
import nltk
nltk.download('punkt')  # This is the Punkt sentence tokenizer.

# This will contain each sentence as a separate document.
sentences_as_documents_essay = []

In [1]:
# def function

In [51]:
def load_and_preprocess(file):
    # Load text
    with open(file, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Apply regex preprocessing (if necessary)
    # regex how to custom it
    regex = r"\n[A-Z]+[\s&]*[A-Z\s]+\s*[0-9]\s*\n\n(.+?)\n\n\n"
    content_each = re.findall(regex, text)
    sentences_as_documents_essay = []
    
    for document in content_each:
    # Tokenize the current document into sentences.
        sentences = nltk.sent_tokenize(document)
        sentences_as_documents_essay.extend(sentences)
    
    return sentences_as_documents_essay  # Or whatever is the processed format

def extract_topics(processed_text):
    #how to find the approrate stopwords
    custom_stopwords=['berenices','lares','penates','inhabitants','city','cities','diomira', 'isidora', 'dorothea', 'zaira', 'anastasia', 'tamara', 'zora', 'despina', 'zirma', 'isaura', 'maurilia', 'fedora', 'zoe', 'zenobia', 'euphemia', 'zobeide', 'hypatia', 'armilla', 'chloe', 'valdrada', 'olivia', 'sophronia', 'eutropia', 'zemrude', 'aglaura', 'octavia', 'ersilia', 'baucis', 'leandra', 'melania', 'esmeralda', 'phyllis', 'pyrrha', 'adelma', 'eudoxia', 'moriana', 'clarice', 'eusapia', 'beersheba', 'leonia', 'irene', 'argia', 'thekla', 'trude', 'olinda', 'laudomia', 'perinthia', 'procopia', 'raissa', 'andria', 'cecilia', 'marozia', 'penthesilea', 'theodora', 'berenice']
    all_stopwords=list(ENGLISH_STOP_WORDS)+ custom_stopwords
    vectorizer_model = CountVectorizer(stop_words= all_stopwords ) 

    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
    representation_model = KeyBERTInspired()
    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = sentence_model.encode(processed_text, show_progress_bar=True)
    umap_model = UMAP(n_neighbors=15, n_components=5, 
                      min_dist=0.0, metric='cosine', random_state=41)

    topic_model = BERTopic(vectorizer_model=vectorizer_model,
                          ctfidf_model=ctfidf_model,
                          representation_model=representation_model,
                          embedding_model=sentence_model,
                          umap_model=umap_model)
    topics, probs = topic_model.fit_transform(processed_text)
    
    df = pd.DataFrame({"Document": processed_text, "Topic": topics})
    
    document_topic_pairs = [(doc, topic) for doc, topic in zip(df['Document'], df['Topic'])]
    
    return document_topic_pairs

def BERTopic_result(file):
    load_and_preprocess(file)
    return extract_topics(load_and_preprocess)
